In [9]:
import pandas as pd
import numpy as np
import json

In [18]:
file = open('Data/line-bot-chrins-export.json', encoding='utf-8')

In [19]:
data = json.load(file)

In [23]:
del data['bmi']

In [27]:
expenses_df = pd.DataFrame.from_dict(data['expenses']).T

In [28]:
expenses_df

,date,money,text
-LqReJpCP9lY2jAui0C8,2019/10/1,12,นมช็อคโกแลต
-LqReMSs0GuMqp-P-sMj,2019/10/1,35,ข้าวเที่ยง
-LqReMw0gwcFp1uB-W2J,2019/10/1,5,น้ำกระเจี๊ยบ
-LqReO0DIuuRn7C-KCA3,2019/10/1,35,ชานม
-LqRePWEGGa8jbdZF1Bn,2019/10/1,50,ข้าวเย็น
...,...,...,...
-M47crdazQX6TB0bs7fx,2020/4/5,50,กะเพรา
-M47cu2KhdKQCli9dL4Q,2020/4/5,14,น้ำ
-M49UnAlyLr-q0GQv0xy,2020/4/5,5768,ค่าห้อง
-M4m1V4K4tfk3fVuGDef,2020/4/13,145,ค่า Netflix
